# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
#from google.colab import userdata
#userdata.get('HF_TOKEN')

# Not needed as will be running it on paperspace

In [1]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-ghyy38aw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-ghyy38aw
  Resolved https://github.com/huggingface/accelerate.git to commit f4ee5a2dc77d5b6bfc889d0b74fbc0678ae56119
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 7.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 40.4 MB/s eta 0:00:00
  Created wheel for accelerate: filename=accelerate-0.35.0.dev0-py3-none-any.whl size=331201 sha256=2eeab1ee2218067e08c86d3b006d4e5ff28292caf79dfe8429ada50b6c7ce2c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-o0e869ny/wheels/23/11/1b/0ca34d88046a6046afe81294675e7c45458915c052f48a8450
Successfully built accelerate
  Attempting uninstall: safetensors
    Found exist

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [3]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [4]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [5]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [7]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [13]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample rows pokemon:
    125 Charmander  Charizard
    268 Cubone      Marowak
    ...
             
    Sample rows types:
    1   fire    125
    2   earth   268
    ...
             
    Sample rows attacks:
    1   fire wheel      10  5   689
    2   elektroshock    15  20  189
    ...

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [14]:
sp_nl2sql = sp_nl2sql.format(question="What is the correct command to determine the attack with the highest attack strength?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample rows pokemon:
    125 Charmander  Charizard
    268 Cubone      Marowak
    ...
             
   

In [15]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

2024-10-09 13:05:26.934579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 13:05:26.934660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 13:05:26.940176: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 13:05:26.956530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 13:05:28.303626: W tensorflow/compiler/tf2

In [16]:
#Empty the cache in order to do more calls without problems.
torch.cuda.empty_cache()

In [17]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT attacks.name, attacks.level, attacks.attack_strength FROM attacks ORDER BY attacks.level DESC, attacks.attack_strength DESC LIMIT 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [18]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample rows pokemon:
    125 Charmander  Charizard
    268 Cubone      Marowak
    ...
             
    Sample rows types:
    1   fire    125
    2   earth   268
    ...
             
    Sample rows attacks:
    1   fire wheel      10  5   689
    2   elektroshock    15  20  189
    ...

    ### Response
    Query: What is the correct command to determine the attack with the highest attack strength?
    Answer: 
    SELECT * 
    FROM attacks 
    ORDER BY attack_strength DESC 
    LIMIT 1;

    Query: What is the correct command to determine the pokemon that can use this attack?
    Answer:
    SELECT p.name AS pokemon_name
    FROM pokemon p
    JOIN attacks a ON p.pokemon_id = a.pokemon_id
    WHERE a.attack_id = (SELECT attack_id FROM attacks ORDER BY attack_strength DESC LIMIT 1);
                    
    Query: What is the correct command to determine what type of attack it is? 
    Answer:                
    SELECT t.name AS attack_type
    FROM types t
    JOIN attacks a ON t.pokemon_id = a.pokemon_id
    WHERE a.attack_id = (SELECT attack_id FROM attacks ORDER BY attack_strength DESC LIMIT 1)

    `{question}`:
    ```sql3
    """


In [19]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample 

In [22]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [23]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT first_name, last_name FROM employees WHERE salary = (SELECT MAX(salary) FROM employees);


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [24]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample rows pokemon:
    125 Charmander  Charizard
    268 Cubone      Marowak
    ...
             
    Sample rows types:
    1   fire    125
    2   earth   268
    ...
             
    Sample rows attacks:
    1   fire wheel      10  5   689
    2   elektroshock    15  20  189
    ...
    
    ### Samples
    
    Query: What is the correct command to determine the attack with the highest attack strength?
    Answer: 
    SELECT * 
    FROM attacks 
    ORDER BY attack_strength DESC 
    LIMIT 1;

    Query: What is the correct command to determine the pokemon that can use this attack?
    Answer:
    SELECT p.name AS pokemon_name
    FROM pokemon p
    JOIN attacks a ON p.pokemon_id = a.pokemon_id
    WHERE a.attack_id = (SELECT attack_id FROM attacks ORDER BY attack_strength DESC LIMIT 1);
                    
    Query: What is the correct command to determine what type of attack it is? 
    Answer:                
    SELECT t.name AS attack_type
    FROM types t
    JOIN attacks a ON t.pokemon_id = a.pokemon_id
    WHERE a.attack_id = (SELECT attack_id FROM attacks ORDER BY attack_strength DESC LIMIT 1)

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [25]:
sp_nl2sql3 = sp_nl2sql3b.format(question="What is the pokemon with the highest number of attacks?")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample r

In [26]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [27]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT p.name AS pokemon_name, COUNT(a.attack_id) AS number_of_attacks
    FROM pokemon p
    JOIN types t ON p.pokemon_id = t.pokemon_id
    JOIN attacks a ON t.type_id = a.pokemon_id
    GROUP BY p.name
    ORDER BY number_of_attacks DESC
    LIMIT 1;


#Now the question in German.


In [28]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Welches Pokemon hat die höchste Anzahl Attacken?")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE pokemon(
    pokemon_id      int not null,
    name            varchar,
    evolved state   varchar         
    )
    
    CREATE TABLE types(
    type_id         int not null,
    name            varchar,
    pokemon_id      int not null
    )
    
    CREATE TABLE attacks(
    attack_id           int not null,
    name                varchar,
    attack_strength     int not null,
    level               int not null,
    pokemon_id          int not null
    )
             
    Sample r

In [29]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [30]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT p.name AS pokemon_name, a.name AS attack_name, t.name AS attack_type
    FROM pokemon p
    JOIN attacks a ON p.pokemon_id = a.pokemon_id
    JOIN types t ON a.type_id = t.type_id
    WHERE a.attack_id = (SELECT attack_id FROM attacks ORDER BY attack_strength DESC LIMIT 1)
    ORDER BY p.pokemon_id NULLS LAST;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# Observations
- The model gave a correct but long and unformatted answer when it wasn't given examples (see 1. below)
- When trying the prompt with a few examples, I used an unrelated question (employee salaries) that could not be answerd with the tables given. The model still found a good answer, but referred to an imaginary table with employee information. This shows that it prioritized answering the question over considering the table given. The query sill wasn't formatted correctly though
- When the examples were given in the seperate "samples" section, the answer was formatted for easy reading (multi-line). It also used abbreviations for the tables to make the query shorter
- When the prompt question was given in German, the answer was still in english and formatted and used abbreviations for the tables. However, the answer was not correct. It was asked to identify the pokemon with the highest amount of attacks (same as 3.), but the model hallucinated as it tried joining two tables on "attacks.type_id" which is a column that does not exist in the attacks table. Also, the model tried to order by pokemon_id, which would not be necessary in this question

# Answers (for reference)
    1. Answer with no shots: SELECT attacks.name, attacks.level, attacks.attack_strength FROM attacks ORDER BY attacks.level DESC, attacks.attack_strength DESC LIMIT 1;

    2. Answer with shots but unrelated question (best paid employee): SELECT first_name, last_name FROM employees WHERE salary = (SELECT MAX(salary) FROM employees);

    3. Answer with shots in seperate "samples" section: 
        SELECT p.name AS pokemon_name, COUNT(a.attack_id) AS number_of_attacks
            FROM pokemon p
            JOIN types t ON p.pokemon_id = t.pokemon_id
            JOIN attacks a ON t.type_id = a.pokemon_id
            GROUP BY p.name
            ORDER BY number_of_attacks DESC
            LIMIT 1;

    4. Answer to prompt in German: 
        SELECT p.name AS pokemon_name, a.name AS attack_name, t.name AS attack_type
            FROM pokemon p
            JOIN attacks a ON p.pokemon_id = a.pokemon_id
            JOIN types t ON a.type_id = t.type_id
            WHERE a.attack_id = (SELECT attack_id FROM attacks ORDER BY attack_strength DESC LIMIT 1)
            ORDER BY p.pokemon_id NULLS LAST;